In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import math
import glob
import time
from datetime import date
from typing import Tuple, Union

import numpy as np
import pandas as pd
import pydicom
from tqdm import tqdm_notebook as tqdm
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

2024-12-17 15:07:16.175911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734448036.196449  186713 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734448036.202844  186713 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#--------------------- for working with directory ----------------------------
ROOT_DATA_PATH='/workspace/datasets'
DATASET_PATH=f'{ROOT_DATA_PATH}/rsna-intracranial-hemorrhage-detection'

TRAIN_DATA_PATH=f'{DATASET_PATH}/stage_2_train'
TRAIN_METADATA_PATH=f'{DATASET_PATH}/processed_metadata/train_metadata_df.pkl'

SEED = 666

In [3]:
#seeding
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
class ImagePreparationUtil:
    @staticmethod
    def correct_dcm(dcm: pydicom.FileDataset):
        x = dcm.pixel_array
        x = x + 1000
        px_mode = 4096
        x[x >= px_mode] = x[x >= px_mode] - px_mode
        dcm.PixelData = x.tobytes()
        dcm.RescaleIntercept = -1000
        
    @staticmethod
    def window_image(dcm: pydicom.FileDataset, window_center: int, window_width: int):
        if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
            ImagePreparationUtil.correct_dcm(dcm)
        
        # Pixel to Hounsfield Unit (HU)
        # HU=(Pixel Value×RescaleSlope)+RescaleIntercept
        img = dcm.pixel_array
        img = img * dcm.RescaleSlope + dcm.RescaleIntercept 
        img_min = window_center - window_width // 2
        img_max = window_center + window_width // 2
        img = np.clip(img, img_min, img_max)
        
        return img
    
    @staticmethod
    def get_windowed_image(dcm: pydicom.FileDataset, window: Union['brain', 'subdural', 'soft'] = 'brain') -> np.ndarray: # type: ignore
        im = None
        match window:
            case 'brain':
                brain_img = ImagePreparationUtil.window_image(dcm, 40, 80)
                brain_img = (brain_img - 0) / 80
                im = brain_img
            case 'subdural':
                subdural_img = ImagePreparationUtil.window_image(dcm, 80, 200)
                subdural_img = (subdural_img - (-20)) / 200
                im = subdural_img
            case 'soft':
                soft_img = ImagePreparationUtil.window_image(dcm, 40, 380)
                soft_img = (soft_img - (-150)) / 380
                im = soft_img
            case _:
                raise ValueError('invalid window argument')
        
        return im
    
    @staticmethod
    def bsb_window(dcm):
        brain_img = ImagePreparationUtil.get_windowed_image(dcm, window='brain')
        subdural_img = ImagePreparationUtil.get_windowed_image(dcm, window='subdural')
        soft_img = ImagePreparationUtil.get_windowed_image(dcm, window='soft')

        bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

        return bsb_img
    
    @staticmethod
    def read(path, resize: Tuple[int, int]):
        img = None
        try:
            dcm = pydicom.dcmread(path)
            img = ImagePreparationUtil.bsb_window(dcm)
        except Exception as e:
            print('\nWarning:', e.__class__.__name__, f'for {path} Replacing with zeros image')
            img = np.zeros((*tuple(resize[:2]), 3), dtype=np.float32)
        
        if resize is not None:
            img = cv2.resize(img, resize[:2], interpolation=cv2.INTER_LINEAR)
            
        return img.astype(np.float32)

In [5]:
class TrainDataGenerator(keras.utils.Sequence):
    def __init__(self, img_ids, labels, img_dir, img_shape, num_classes, batch_size, under_sampling=False, shuffle_on_epoch_end=True, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.img_ids = img_ids
        self.labels = labels
        self.img_dir = img_dir
        self.img_shape = img_shape
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.under_sampling = under_sampling
        self.shuffle_on_epoch_end = shuffle_on_epoch_end
        self.on_epoch_end()
    
    def __len__(self):
        return math.ceil(len(self.img_ids) / self.batch_size)

    def __getitem__(self, index):
        low = index*self.batch_size
        high = min(low + self.batch_size, len(self.img_ids))
        indices = self.indices[low:high]
        X,y = self.__data_generation(indices)

        return X,y
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.img_ids))
        
        if self.under_sampling:
            keep_prob = self.labels.iloc[:,0].map({0: 0.35, 1: 0.5})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = self.indices[keep]
            
        if self.shuffle_on_epoch_end:
            np.random.shuffle(self.indices)

        
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_shape))
        y = np.empty((self.batch_size, self.num_classes), dtype=np.float32)
        
        for i, img_idx in enumerate(indices):
            img_id = self.img_ids[img_idx]
            img_path = f'{self.img_dir}/{img_id}.dcm'
            img = ImagePreparationUtil.read(img_path, self.img_shape[:2])
            X[i,] = img
            y[i,] = self.labels.iloc[img_idx].values
            
        return X,y
         

In [6]:
df = pd.read_pickle(TRAIN_METADATA_PATH)
test_df_neg = df[df['Label']['any'] == 0].sample(n=200)
test_df_pos = df[df['Label']['any'] == 1].sample(n=200)
test_df = pd.concat([test_df_pos, test_df_neg]).sample(frac=1)
df = df.drop(test_df.index)
df

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_99196d0ab     1        0                0                0            0   
ID_2b0190b58     0        0                0                0            0   
ID_056e14224     1        1                0                0            0   
ID_525e72262     0        0                0                0            0   
ID_b00eddf10     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_21e825c20     0        0                0                0            0   
ID_85141c704     1        0                1                0            0   
ID_b4add57dd     0        0                0                0            0   
ID_02f0d7dbb     0        0                0                0            0   
ID_7d29263ce     1        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_99196d0ab        1  
ID_2b0190b58        0  
ID_056e14224        0  
ID_525e72262        0  
ID_b00eddf10        0  
...               ...  
ID_21e825c20        0  
ID_85141c704        1  
ID_b4add57dd        0  
ID_02f0d7dbb        0  
ID_7d29263ce        1  

[263606 rows x 6 columns]

In [7]:
df['Label']['any'].value_counts()

any
0    146510
1    117096
Name: count, dtype: int64

In [8]:
test_df['Label']['any'].value_counts()

any
1    200
0    200
Name: count, dtype: int64

In [9]:
class LearningRateLogger(keras.callbacks.Callback):        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        lr = self.model.optimizer.learning_rate
        if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
            lr = lr(self.model.optimizer.iterations)
        logs['learning_rate'] = float(tf.keras.backend.get_value(lr))
        print(f"Epoch {epoch + 1}: Learning rate is {logs['learning_rate']}")
        
class DeepModel():
    def __init__(self, base,input_shape, batch_size, num_epochs, num_classes, learning_rate, decay_rate, decay_steps, base_name='base_model', weights='imagenet', freeze_base=False, saving_path=None, verbose=1):
        self.base = base
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.base_name = base_name
        self.weights = weights
        self.freeze_base = freeze_base
        self.saving_path = saving_path
        self.verbose = verbose
        self._build()
        self._create_saving_path_if_none()
    
    def _build(self):
        K.clear_session()
        base = self.base(include_top=False, weights=self.weights, pooling='avg', input_shape=self.input_shape)
        base.trainable = not self.freeze_base
            
        x = base.output
        x = layers.Dropout(0.2)(x)
        out = layers.Dense(self.num_classes, activation='sigmoid', name='dense_output')(x)
        
        self.model = keras.models.Model(inputs=base.input, outputs=out)
        
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=self.learning_rate,
            decay_steps=self.decay_steps,
            decay_rate=self.decay_rate,
            staircase=True,
        )
        
        self.model.compile(
            loss=keras.losses.BinaryCrossentropy(), 
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=[
                keras.metrics.TruePositives(name='tp'),
                keras.metrics.TrueNegatives(name='tn'),
                keras.metrics.FalsePositives(name='fp'),
                keras.metrics.FalseNegatives(name='fn'),
                keras.metrics.BinaryAccuracy(name='binary_acc'), 
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall'),
                keras.metrics.AUC(name='auc_roc', curve='ROC', multi_label=True, num_labels=self.num_classes),
                keras.metrics.AUC(name='auc_pr', curve='PR', multi_label=True, num_labels=self.num_classes),
                keras.metrics.F1Score(name='f1_score', average='weighted'),                
            ]
        )
        
    def _create_saving_path_if_none(self):
        if self.saving_path is None:
            print('saving_path is None. Automatically create path for save training utilities')
            saving_path = os.path.join('saved_model', self.base.__name__)
            os.makedirs(saving_path, exist_ok=True)
            self.saving_path = saving_path
            print(self.saving_path, 'created for saving_path')
        
    def fit(self, train_df, valid_df):
        train_data_gen = TrainDataGenerator(
            img_ids=train_df.index, 
            labels=train_df,
            img_dir=TRAIN_DATA_PATH,
            img_shape=self.input_shape,
            num_classes=self.num_classes,
            batch_size=self.batch_size,
            under_sampling=False,
            workers=128,
            use_multiprocessing=True
        )
        
        valid_data_gen = TrainDataGenerator(
            img_ids=valid_df.index, 
            labels=valid_df,
            img_dir=TRAIN_DATA_PATH,
            img_shape=self.input_shape,
            num_classes=self.num_classes,
            batch_size=self.batch_size,
            under_sampling=False,
            workers=128,
            use_multiprocessing=True
        )
        
        curr_time = int(time.time())
        callbacks = [
            keras.callbacks.ModelCheckpoint(filepath='%s/{epoch:02d}-%s.weights.h5' % (self.saving_path, curr_time), monitor='val_loss', mode='min', verbose=self.verbose, save_weights_only=True, save_best_only=True),
            LearningRateLogger(),
            keras.callbacks.EarlyStopping(verbose=1, restore_best_weights=True, monitor='val_loss', patience=5, mode='min'),
            keras.callbacks.CSVLogger(filename='%s/%s.csv'%(self.saving_path, f'{curr_time}-train_log'))
        ]
        return self.model.fit(
            x=train_data_gen,
            validation_data=valid_data_gen,
            callbacks=callbacks,
            epochs=self.num_epochs,
            verbose=self.verbose
        )
    
    def save(self, filename, overwrite=True):
        _, ext = os.path.splitext(filename)
        if ext != '.keras':
          filename = f'{filename}.keras'
          
        p = os.path.join(self.saving_path, filename)
        self.model.save(p, overwrite=overwrite)
        print('model saved to:', p)
        
    def load_weights(self, path, *args, **kwargs):
        self.model.load_weights(path, *args, **kwargs)
        
    def load(self, path, *args, **kwargs):
        self.model = keras.models.load_model(path, *args, **kwargs)
        

In [10]:
EPOCH = 15
INPUT_SHAPE = (256, 256, 3)
BATCH_SIZE = 32
N_CLASSES = 6

In [11]:
msss = MultilabelStratifiedShuffleSplit(n_splits=EPOCH, test_size=0.15, random_state=SEED)
X = df.index
Y = df['Label'].values

msss_splits = next(msss.split(X, Y))
train_idx = msss_splits[0]
valid_idx = msss_splits[1]

In [12]:
steps_per_epoch = math.ceil(len(train_idx)/BATCH_SIZE)

In [13]:
saving_path = 'saved_model/EfficientNetB4_V2_image_input'
os.makedirs(saving_path, exist_ok=True)

model = DeepModel(
    base=keras.applications.EfficientNetB4,
    weights='imagenet',
    freeze_base=False,
    input_shape=INPUT_SHAPE, 
    batch_size=BATCH_SIZE, 
    num_epochs=EPOCH,
    num_classes=N_CLASSES,
    learning_rate=0.000125, 
    decay_rate=0.8, 
    decay_steps=2*steps_per_epoch,
    base_name='efficientnetb4',
    saving_path=saving_path
)

I0000 00:00:1734448053.525576  186713 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8225 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.10gb, pci bus id: 0000:07:00.0, compute capability: 8.0


In [ ]:
np.random.shuffle(train_idx)

hist = model.fit(df.iloc[train_idx], df.iloc[valid_idx])
model.save('Best_EfficientNetB4_V2_image_input.keras')

Epoch 1/15


I0000 00:00:1734448147.683356  187258 service.cc:148] XLA service 0x7fbe000035f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734448147.683416  187258 service.cc:156]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 1g.10gb, Compute Capability 8.0
I0000 00:00:1734448156.180971  187258 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1734448217.321975  187258 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7003/7003 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - auc_pr: 0.7041 - auc_roc: 0.9114 - binary_acc: 0.9078 - f1_score: 0.3039 - fn: 37672.8281 - fp: 17121.5840 - loss: 0.2258 - precision: 0.7978 - recall: 0.6473 - tn: 535854.1875 - tp: 81735.3984
Epoch 1: val_loss improved from inf to 0.37568, saving model to saved_model/EfficientNetB4_V2_image_input/01-1734448057.weights.h5
Epoch 1: Learning rate is 0.0001250000059371814
7003/7003 ━━━━━━━━━━━━━━━━━━━━ 3107s 421ms/step - auc_pr: 0.7041 - auc_roc: 0.9114 - binary_acc: 0.9078 - f1_score: 0.3039 - fn: 37676.9766 - fp: 17123.6484 - loss: 0.2258 - precision: 0.7978 - recall: 0.6473 - tn: 535931.0625 - tp: 81748.2969 - val_auc_pr: 0.6471 - val_auc_roc: 0.8470 - val_binary_acc: 0.8746 - val_f1_score: 0.3022 - val_fn: 28789.0000 - val_fp: 976.0000 - val_loss: 0.3757 - val_precision: 0.9321 - val_recall: 0.3177 - val_tn: 194142.0000 - val_tp: 13405.0000 - learning_rate: 1.2500e-04
Epoch 2/15
7003/7003 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - auc_pr: 0.90

In [18]:
logpath = os.path.join(model.saving_path, '1734448057-train_log.csv') 
train_history = pd.read_csv(logpath)
train_history['epoch'] = train_history['epoch']+1
train_history.to_csv(logpath, index=False)

In [19]:
train_history

,epoch,auc_pr,auc_roc,binary_acc,f1_score,fn,fp,learning_rate,loss,precision,...,val_auc_roc,val_binary_acc,val_f1_score,val_fn,val_fp,val_loss,val_precision,val_recall,val_tn,val_tp
0,1,0.804329,0.946404,0.926891,0.319842,66716.0,31583.0,0.000125,0.184763,0.844901,...,0.846988,0.874574,0.302227,28789.0,976.0,0.375677,0.932133,0.317699,194142.0,13405.0
1,2,0.913539,0.975117,0.951159,0.356002,42275.0,23394.0,0.000100,0.126985,0.893600,...,0.750989,0.848697,0.338254,35607.0,299.0,0.610051,0.956604,0.156192,194815.0,6591.0
2,3,0.951139,0.986248,0.964419,0.379337,29537.0,18303.0,0.000100,0.093257,0.919556,...,0.902070,0.912512,0.371446,18202.0,2560.0,0.298950,0.903589,0.568622,192557.0,23993.0
3,4,0.970556,0.992120,0.973761,0.394798,21156.0,14124.0,0.000080,0.069433,0.939052,...,0.929897,0.928537,0.393722,12161.0,4798.0,0.227655,0.862265,0.711811,190316.0,30037.0
4,5,0.985032,0.996266,0.982453,0.408839,13775.0,9817.0,0.000080,0.046643,0.958190,...,0.724002,0.864002,0.343975,31325.0,949.0,0.951010,0.919712,0.257631,194167.0,10871.0
5,6,0.990906,0.997713,0.987037,0.417757,9857.0,7571.0,0.000064,0.034770,0.967984,...,0.966596,0.951390,0.406147,6524.0,5012.0,0.163734,0.876828,0.845414,190097.0,35679.0
6,7,0.995015,0.998715,0.991049,0.426655,6718.0,5313.0,0.000064,0.024324,0.977615,...,0.851835,0.891051,0.397170,23157.0,2698.0,0.480995,0.875903,0.451256,192414.0,19043.0
7,8,0.996248,0.999078,0.992629,0.433511,5499.0,4404.0,0.000051,0.020075,0.981469,...,0.872857,0.917939,0.419859,16474.0,3000.0,0.464225,0.895576,0.609649,192109.0,25729.0
8,9,0.997861,0.999417,0.994695,0.444742,3934.0,3190.0,0.000051,0.014753,0.986597,...,0.840449,0.908842,0.407943,19627.0,2006.0,0.580511,0.918409,0.534982,193099.0,22580.0
9,10,0.998154,0.999494,0.995197,0.448080,3495.0,2954.0,0.000041,0.013289,0.987600,...,0.921558,0.939902,0.452039,11679.0,2583.0,0.356227,0.921976,0.723253,192528.0,30522.0


In [ ]:

# Set seaborn theme
sns.set_theme(style="whitegrid")

# Create the figure and axes
fig, axes = plt.subplots(2, 3, figsize=(18, 8), sharey=False)
fig.suptitle("Training History", fontsize=18, fontweight="bold", color="darkblue")

# 1st subplot: AUC-ROC
sns.lineplot(data=train_history, x='epoch', y='auc_roc', label='Train AUC-ROC', ax=axes[0, 0], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_auc_roc', label='Validation AUC-ROC', ax=axes[0, 0], color='orange', linewidth=2, marker='o')
# lambda arguments : expression
# axes[0].annotate(text='best', xy=best('epoch', 'val_auc_roc'))
axes[0, 0].set_ylabel("AUC-ROC", fontsize=14)
axes[0, 0].set_title("AUC-ROC vs. Epochs", fontsize=14)
axes[0, 0].legend(loc="lower right", fontsize=10)

# 2nd subplot: Loss
sns.lineplot(data=train_history, x='epoch', y='loss', label='Train Loss', ax=axes[0, 1], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_loss', label='Validation Loss', ax=axes[0, 1], color='orange', linewidth=2, marker='o')
# axes[1].annotate(text='best', xy=best('epoch', 'val_loss'))
axes[0, 1].set_ylabel("Loss", fontsize=14)
axes[0, 1].set_title("Loss vs. Epochs", fontsize=14)
axes[0, 1].legend(loc="lower left", fontsize=10)

# 3rd subplot: Accuracy
sns.lineplot(data=train_history, x='epoch', y='binary_acc', label='Train Accuracy', ax=axes[0, 2], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_binary_acc', label='Validation Accuracy', ax=axes[0, 2], color='orange', linewidth=2, marker='o')
# axes[2].annotate(text='best', xy=best('epoch', 'val_loss'))
axes[0, 2].set_ylabel("Accuracy", fontsize=14)
axes[0, 2].set_title("Accuracy vs. Epochs", fontsize=14)
axes[0, 2].legend(loc="lower right", fontsize=10)

# 4th subplot: AUC-PR
sns.lineplot(data=train_history, x='epoch', y='auc_pr', label='Train AUC-PR', ax=axes[1, 0], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_auc_pr', label='Validation AUC-PR', ax=axes[1, 0], color='orange', linewidth=2, marker='o')
# axes[2].annotate(text='best', xy=best('epoch', 'val_loss'))
axes[1, 0].set_ylabel("AUC-PR", fontsize=14)
axes[1, 0].set_title("AUC-PR vs. Epochs", fontsize=14)
axes[1, 0].legend(loc="lower right", fontsize=10)

# 5th subplot: F1-score
sns.lineplot(data=train_history, x='epoch', y='precision', label='Precision', ax=axes[1, 1], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_precision', label='Precision validation', ax=axes[1, 1], color='orange', linewidth=2, marker='o')
# axes[2].annotate(text='best', xy=best('epoch', 'val_loss'))
axes[1, 1].set_ylabel("Precision", fontsize=14)
axes[1, 1].set_title("Precision vs. Epochs", fontsize=14)
axes[1, 1].legend(loc="lower right", fontsize=10)

# 6th subplot: F1-score
sns.lineplot(data=train_history, x='epoch', y='recall', label='Recall', ax=axes[1, 2], color='blue', linewidth=2, marker='o')
sns.lineplot(data=train_history, x='epoch', y='val_recall', label='Recall validation', ax=axes[1, 2], color='orange', linewidth=2, marker='o')
# axes[2].annotate(text='best', xy=best('epoch', 'val_loss'))
axes[1, 2].set_ylabel("Recall", fontsize=14)
axes[1, 2].set_title("Recall vs. Epochs", fontsize=14)
axes[1, 2].legend(loc="lower right", fontsize=10)


# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 1])  # Leave space for suptitle
plt.show()